In [53]:
import findspark
findspark.init()

In [54]:
from pyspark import SparkConf, SparkContext

In [55]:
conf = SparkConf().setMaster("local[4]").setAppName("BroadcastVariablesOps")
sc = SparkContext(conf=conf).getOrCreate()

In [34]:
# products.csv dosyasını okuyup (urün_id, ürün_adı) döndüren fonksiyon
def read_products():
    products_text_wrapper = open("D:/Datasets/retail_db/products.csv", "r",encoding='utf-8')
    # satır satır okuyalım
    products = products_text_wrapper.readlines()
    product_id_and_name = []
    
    for line in products:
        if "productName" not in line:
            product_id = line.split(",")[0]
            product_name = line.split(",")[2]
            product_id_and_name.append((product_id,product_name))
    
    return product_id_and_name
    

In [35]:
products = read_products()

In [36]:
broadcast_product = sc.broadcast(products)

In [38]:
broadcast_product.value[:10]

[('1', 'Quest Q64 10 FT. x 10 FT. Slant Leg Instant U'),
 ('2', "Under Armour Men's Highlight MC Football Clea"),
 ('3', "Under Armour Men's Renegade D Mid Football Cl"),
 ('4', "Under Armour Men's Renegade D Mid Football Cl"),
 ('5', 'Riddell Youth Revolution Speed Custom Footbal'),
 ('6', "Jordan Men's VI Retro TD Football Cleat"),
 ('7', 'Schutt Youth Recruit Hybrid Custom Football H'),
 ('8', "Nike Men's Vapor Carbon Elite TD Football Cle"),
 ('9', 'Nike Adult Vapor Jet 3.0 Receiver Gloves'),
 ('10', "Under Armour Men's Highlight MC Football Clea")]

# order_item okuma ve rdd oluşturma

In [41]:
order_items_rdd = sc.textFile("D:Datasets/retail_db/order_items.csv") \
.filter(lambda x: "orderItemName" not in x)

In [42]:
order_items_rdd.take(5)

['1,1,957,1,299.98,299.98',
 '2,2,1073,1,199.99,199.99',
 '3,2,502,5,250.0,50.0',
 '4,2,403,1,129.99,129.99',
 '5,4,897,2,49.98,24.99']

# order_items pair_rdd yapma 

In [43]:
def make_order_items_pair_rdd(line):
    order_item_name = line.split(",")[0]
    order_item_order_id = line.split(",")[1]
    order_item_product_id = int(line.split(",")[2])
    order_item_quantity = line.split(",")[3]
    order_item_sub_total = float(line.split(",")[4])
    order_item_product_price = line.split(",")[5]
    
    return (order_item_product_id, order_item_sub_total)
    

In [44]:
order_item_pair_rdd = order_items_rdd.map(make_order_items_pair_rdd)

In [45]:
order_item_pair_rdd.take(6)

[(957, 299.98),
 (1073, 199.99),
 (502, 250.0),
 (403, 129.99),
 (897, 49.98),
 (365, 299.95)]

# order_items ile broadcast variable olan products birleştirme

In [46]:
order_item_pair_rdd.reduceByKey(lambda x,y: x+y) \
.map(lambda x: (x[1], x[0])) \
.sortByKey(False) \
.map(lambda x: (x[1], x[0])) \
.take(5)

[(1004, 6929653.499999708),
 (365, 4421143.019999639),
 (957, 4118425.419999785),
 (191, 3667633.1999997487),
 (502, 3147800.0)]

In [47]:
sorted_orders = order_item_pair_rdd.reduceByKey(lambda x,y: x+y) \
.map(lambda x: (x[1], x[0])) \
.sortByKey(False) \
.map(lambda x: (x[1], x[0]))

In [49]:
sorted_orders_with_product_name = sorted_orders.map(lambda x: (broadcast_product.value[x[0]], x[1]))

In [51]:
sorted_orders_with_product_name.take(10)

[(('1005', 'Diamond Infinite Edge RTH Compound Bow Packag'),
  6929653.499999708),
 (('366', 'Gazelle Supreme Glider'), 4421143.019999639),
 (('958', 'Nishiki Adult Montour Hybrid Bike 2014'), 4118425.419999785),
 (('192', '"Nike Women\'s Pro Core 3"" Compression Shorts"'),
  3667633.1999997487),
 (('503', 'Nike Youth USA Away Stadium Replica Soccer Je'), 3147800.0),
 (('1074', 'Field & Stream Eagle Talon Kayak'), 3099844.999999871),
 (('404', "Nike Men's Vapor Carbon Elite TD Football Cle"),
  2891757.5399998166),
 (('1015', 'GoPro HERO3+ Black Edition Camera'), 2888993.9399996493),
 (('628', '"Under Armour Elevate 22 oz. Leak-Proof Water "'),
  1269082.649999932),
 (('566', "adidas Men's Germany Black/Red Away Match Soc"), 67830.0)]

In [52]:
sc.stop()

In [59]:
products_text_file2 = open("D:/Datasets/retail_db/products.csv", "r",encoding="utf-8")
products_text_file2

<_io.TextIOWrapper name='D:/Datasets/retail_db/products.csv' mode='r' encoding='utf-8'>